In [ ]:

import os
import warnings
import pandas as pd       
import requests

from automlwrapper import AutoMLWrapper
from sedarapi import SedarAPI


In [ ]:

experiment_id = "{experiment_id}"
remote_server_uri = "{os.environ['MLFLOW_TRACKING_URI']}"
mlflow.set_tracking_uri(remote_server_uri) 
mlflow.set_experiment(experiment_id=experiment_id)


In [ ]:

base_url = "http://192.168.220.107:5000"
email = "admin"
password = "admin"

sedar = SedarAPI(base_url)
sedar.connection.logger.setLevel("INFO") 

sedar.login(email, password)

ws = sedar.get_workspace("13b4787c3e454649aa05a4cd680edc37") 
dataset = ws.get_dataset("986f2e837ca44f3e8c0ee7d2dc0c4287")

query_string = f"SELECT * FROM {dataset.id}"
try:
    query_result = dataset.query_sourcedata(query_string)
    glass_schema = pd.DataFrame.from_dict(query_result['body'])
except Exception as e:
    print(e)


In [ ]:

wrapper = AutoMLWrapper('autokeras')
wrapper.Train(
    data=glass_schema,
    target_column='Type',
    task_type='classification',
    data_type='tabular',
    problem_type='multiclass',
    hyperparameters={}
)
wrapper.MlflowUploadBest({})


In [ ]:

    data = {
    "notebook_id":"36f4628c-ef0c-41e1-9e85-5fed85e346fa",
    "run_id":run_id,
    "experiment_id":experiment_id
   }
res = requests.post(url = "http://192.168.220.107:5000/api/v1/workspaces/13b4787c3e454649aa05a4cd680edc37/add-mlrun/986f2e837ca44f3e8c0ee7d2dc0c4287", data=data)    
